In [97]:
with open('16.txt') as f:
    GRID = [line.strip() for line in f.readlines()]

In [98]:
from typing import Literal

Coord = tuple[int, int]
Direction = Literal['>', '<', '^', 'v']
DIRECTIONS: dict[Direction, Coord] = {
    '>': (1, 0),
    '<': (-1, 0),
    '^': (0, -1),
    'v': (0, +1),
}
# we want to go up and right, so we'll prioritise those first
TURNS: dict[Direction, list[Direction]] = {
    '>': '^v',
    '<': '^v',
    '^': '><',
    'v': '><',
}

def finds(grid, target):
    for y, line in enumerate(grid):
        for x, char in enumerate(line):
            if char == target:
                return x, y

START = finds(GRID, 'S')
END = finds(GRID, 'E')

In [99]:
def update_costs(
    coord: Coord,
    facing: Direction,
    distances: dict[Coord, list[int]],
    cost: int = 0
):
    x, y = coord
    if GRID[y][x] == '#':
        return distances
    if (d := distances.get(coord)):
        if cost > min(d):
            return distances
    distances.setdefault(coord, []).append(cost)
    dx, dy = DIRECTIONS[facing]
    distances = update_costs(
        (x+dx, y+dy), facing, distances, cost+1)
    for d in TURNS[facing]:
        dx, dy = DIRECTIONS[d]
        distances = update_costs(
            (x+dx, y+dy), d, distances, cost+1001)

    return distances

print(min(update_costs(START, '>', {})[END]))

7036


# Part 2: Pathing

## Attempt 1: Very Naïve, No Good Flood Fill
As it turns out, the `cost < min(d)` requirement is not exhaustive – it excludes alternate paths which still fit the bill.

In [100]:
from functools import reduce
from operator import or_


def get_valid_paths(
    coord: Coord,
    facing: Direction,
    cost: int = 0,
    path: list[Coord] = [],
    distances: dict[Coord, list[int]] = dict(),
):
    x, y = coord
    if GRID[y][x] == '#':
        return
    if GRID[y][x] == 'E':
        yield cost, path
    if coord in path:
        return
    path = path + [coord]
    if (d := distances.get(coord)):
        if cost > min(d):
            return distances
    distances.setdefault(coord, []).append(cost)

    dx, dy = DIRECTIONS[facing]
    yield from get_valid_paths(
        (x+dx, y+dy), facing, cost+1, path, distances
    )
    for d in TURNS[facing]:
        dx, dy = DIRECTIONS[d]
        yield from get_valid_paths(
            (x+dx, y+dy), d, cost+1001, path, distances)

paths = list(get_valid_paths(START, '>'))
minscore = min([s for s, _ in paths])
minpaths = [set(x) for l, x in paths if l == minscore]
tiles = reduce(or_, minpaths, set())
grid2 = [list(line) for line in GRID]
for x, y in tiles:
    grid2[y][x] = 'O'
print('\n'.join(''.join(line) for line in grid2))
print(len(tiles) + 1)

###############
#.......#....E#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#.#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############
44


## Attempt 2: Thinking With Portals

In [101]:
from queue import Queue


to_scan: Queue[Coord] = Queue()
to_scan.put(START)
scores: dict[Coord, int] = dict()
scores[START] = 0
while to_scan:
    x, y = to_scan.get()
    for dx, dy in DIRECTIONS.values():
        coord = x+dx, y+dy
        if coord in scores:
            continue
        to_scan.put(coord)
        scores[coord] = scores[x, y] + 1



KeyboardInterrupt: 